<a href="https://colab.research.google.com/github/amimulhasan/Deep-Learning/blob/main/gvasasdhdcjsbd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile

zip_path = '/content/drive/MyDrive/MRI_DATA_IMAGE.zip'
extract_to = 'brain_tumor_data'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Unzipping completed!")

Unzipping completed!


In [4]:
# Install keras-cv if needed for VisionTransformer
# !pip install keras-cv --quiet

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt


In [5]:
img_size = 224
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/brain_tumor_data/MRI_DATA_IMAGE",
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_size, img_size),
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/brain_tumor_data/MRI_DATA_IMAGE",
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_size, img_size),
    batch_size=batch_size
)

class_names = train_ds.class_names
print("Classes:", class_names)


Found 1176 files belonging to 2 classes.
Using 941 files for training.
Found 1176 files belonging to 2 classes.
Using 235 files for validation.
Classes: ['Normal', 'Tumor Patient']


In [6]:
def build_cnn_branch(input_shape):
    inputs = keras.Input(shape=input_shape)
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Flatten()(x)
    cnn_output = layers.Dense(128, activation='relu')(x)
    return keras.Model(inputs, cnn_output)


In [8]:
from transformers import ViTFeatureExtractor, ViTForImageClassification


In [11]:
!pip install keras-cv-attention-models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.6/802.6 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.3 MB/s eta 0:00:00


In [14]:
from keras_cv_attention_models import vit
# vit_model = vit.VisionTransformer(num_classes=10)


In [15]:
!pip install keras-cv --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 34.9 MB/s eta 0:00:00


In [16]:
!pip install keras-cv-attention-models --upgrade


In [20]:
!pip install keras-cv-attention-models --upgrade


In [21]:
!pip install keras-cv-attention-models --upgrade
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras_cv_attention_models import vit

def build_vit_branch(input_shape):
    inputs = keras.Input(shape=input_shape)
    x = vit.VisionTransformer(
        input_shape=input_shape,
        pretrained=None,
        num_classes=0  # no classifier head
    )(inputs)
    x = layers.GlobalAveragePooling2D()(x)
    vit_output = layers.Dense(128, activation='relu')(x)
    return keras.Model(inputs, vit_output)


In [25]:
from keras_cv_attention_models import vit

def build_vit_branch(input_shape):
    inputs = keras.Input(shape=input_shape)
    backbone = vit.ViT(
        input_shape=input_shape,
        pretrained=None,
        num_classes=0   # no classifier head
    )
    x = backbone(inputs)
    x = layers.GlobalAveragePooling2D()(x)
    vit_output = layers.Dense(128, activation='relu')(x)
    return keras.Model(inputs, vit_output)


In [22]:
# from keras_cv.models import VisionTransformer

In [ ]:
# from keras_cv.models import VisionTransformer

# def build_vit_branch(input_shape):
#     vit = VisionTransformer(
#         include_top=False,
#         class_names=None,
#         image_size=input_shape[0],
#         patch_size=16,
#         transformer_layers=8,
#         num_heads=8,
#         projection_dim=64,
#         mlp_head_units=[128],
#         output_pooling="avg"
#     )
#     inputs = keras.Input(shape=input_shape)
#     x = vit(inputs)
#     vit_output = layers.Dense(128, activation='relu')(x)
#     return keras.Model(inputs, vit_output)


In [23]:
def build_lstm_branch(input_shape):
    inputs = keras.Input(shape=input_shape)
    x = layers.Reshape((input_shape[0]*input_shape[1], 3))(inputs)  # shape: (time_steps, features)
    x = layers.LSTM(64, return_sequences=False)(x)
    lstm_output = layers.Dense(128, activation='relu')(x)
    return keras.Model(inputs, lstm_output)


In [28]:
from keras_cv_attention_models import vit

def build_vit_branch(input_shape):
    inputs = keras.Input(shape=input_shape)
    backbone = vit.ViT(
        input_shape=input_shape,
        pretrained=None,
        num_classes=0  # no classifier head
    )
    # Explicitly use a Lambda to stay inside Keras symbolic world
    x = layers.Lambda(lambda inp: backbone(inp))(inputs)
    x = layers.GlobalAveragePooling2D()(x)
    vit_output = layers.Dense(128, activation='relu')(x)
    return keras.Model(inputs, vit_output)


In [29]:
def build_vit_branch(input_shape):
    inputs = keras.Input(shape=input_shape)
    backbone = vit.ViT(input_shape=input_shape, pretrained=None, num_classes=0)
    x = layers.Lambda(lambda inp: backbone(inp))(inputs)
    x = layers.GlobalAveragePooling2D()(x)
    vit_output = layers.Dense(128, activation='relu')(x)
    return keras.Model(inputs, vit_output)


In [30]:
input_shape = (img_size, img_size, 3)
cnn_branch = build_cnn_branch(input_shape)
vit_branch = build_vit_branch(input_shape)
lstm_branch = build_lstm_branch(input_shape)

inputs = keras.Input(shape=input_shape)
cnn_out = cnn_branch(inputs)
vit_out = vit_branch(inputs)
lstm_out = lstm_branch(inputs)

concat = layers.concatenate([cnn_out, vit_out, lstm_out])
x = layers.Dense(64, activation='relu')(concat)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(len(class_names), activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=outputs)


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()


In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20
)


In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title('Accuracy')

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss')
plt.show()
